In [11]:
# source: https://www.manning.com/liveproject/how-to-think-about-manipulating-data

import pandas as pd

# 1. Reading in and Cleaning the Data
url="https://en.wikipedia.org/wiki/NBA_All-Star_Game"
table = pd.read_html(url,header=0,index_col=0,parse_dates=True)
print(type(table)) # <class 'list'> # array of DataFrame
print(len(table)) # 16
print(type(table[2])) # <class 'pandas.core.frame.DataFrame'> # table[2] is already a DataFrame
print(table[2].shape) # (74, 4)
df = table[2].dropna(how='any')
print(df.shape) # (71, 4) # 1951 - 2021; the last three rows (2022 2023 2024) are dropped because there are empty cells
df

<class 'list'>
16
<class 'pandas.core.frame.DataFrame'>
(74, 4)
(71, 4)


,Result,Host arena,Host city,Game MVP
Year,,,,
1951,"East 111, West 94",Boston Garden,"Boston, Massachusetts","Ed Macauley, Boston Celtics"
1952,"East 108, West 91",Boston Garden (2),"Boston, Massachusetts (2)","Paul Arizin, Philadelphia Warriors"
1953,"West 79, East 75",Allen County War Memorial Coliseum,"Fort Wayne, Indiana","George Mikan, Minneapolis Lakers"
1954,"East 98, West 93 (OT)",Madison Square Garden III**,"New York City, New York","Bob Cousy, Boston Celtics"
1955,"East 100, West 91",Madison Square Garden III** (2),"New York City, New York (2)","Bill Sharman, Boston Celtics"
...,...,...,...,...
2017,"West 192, East 182",Smoothie King Center (3),"New Orleans, Louisiana (3)","Anthony Davis, New Orleans Pelicans"
2018[5],"Team LeBron 148, Team Stephen 145",Staples Center (3),"Los Angeles, California (4)","LeBron James (3), Cleveland Cavaliers"
2019,"Team LeBron 178, Team Giannis 164",Spectrum Center,"Charlotte, North Carolina (2)","Kevin Durant (2), Golden State Warriors"


In [12]:
# 2. Keeping Only Data of Interest
def parse_east(line):
    parts = line.split(",")
    parts0 = parts[0].split()
    parts1 = parts[1].split()
    if parts0[0].lower() == "east":
        return int(parts0[1])
    elif parts1[0].lower() == "east":
        return int(parts1[1])

def parse_west(line):
    parts = line.split(",")
    parts0 = parts[0].split()
    parts1 = parts[1].split()
    if parts0[0].lower() == "west":
        return int(parts0[1])
    elif parts1[0].lower() == "west":
        return int(parts1[1])

df_east = df.loc[:,"Result"] # pick a column
df_east = df_east.apply(parse_east) # transform every cell
df_west = df.loc[:,"Result"]
df_west = df_west.apply(parse_west)
del(df["Result"]) # delete a column
del(df["Host arena"])
del(df["Game MVP"])
df.insert(0,"West",df_west) # add a column
df.insert(0,"East",df_east)
# df = df.rename(index=lambda s: s.split("[")[0]) # not useful?
df = df.dropna(how='any') # 5 rows dropped: 2018 - 2021 with names not like "east" "west"; 1999 with canceled info
df["Host city"] = df["Host city"].apply(lambda s: s.strip().split(",")[0])
df # 66 x 3

,East,West,Host city
Year,,,
1951,111.0,94.0,Boston
1952,108.0,91.0,Boston
1953,75.0,79.0,Fort Wayne
1954,98.0,93.0,New York City
1955,100.0,91.0,New York City
...,...,...,...
2013,138.0,143.0,Houston
2014,163.0,155.0,New Orleans
2015,158.0,163.0,New York City


In [24]:
# 3. Getting counts
df['Diff'] = abs(df['East'] - df['West']) # calculating and adding a "Diff" column at the same time
df.to_csv("nba.csv")

dfd = df.groupby('Diff').size().sort_values(ascending=False)
# print(dfd) # 5.0 - 7; ... 43.0 - 1 # count of each diff # not used anywhere else

## question 1. What is the maximum and minimum score difference between all games?
maxdiff = max(df["Diff"])
print("Max difference:" + str(maxdiff))
print(df.loc[df["Diff"] == maxdiff]) # filtering rows on condition
mindiff = min(df["Diff"])
print()
print("Min difference:" + str(mindiff))
print(df.loc[df["Diff"] == mindiff])

dfd

Max difference:43.0
       East  West   Host city  Diff
Year                               
1966  137.0  94.0  Cincinnati  43.0

Min difference:1.0
       East   West   Host city  Diff
Year                                
1965  124.0  123.0   St. Louis   1.0
1971  107.0  108.0   San Diego   1.0
1977  124.0  125.0   Milwaukee   1.0
2001  111.0  110.0  Washington   1.0


Diff
5.0     7
11.0    5
2.0     5
9.0     5
1.0     4
10.0    4
3.0     3
4.0     3
7.0     3
8.0     3
12.0    3
17.0    3
20.0    3
14.0    2
15.0    2
6.0     2
21.0    2
27.0    2
40.0    1
16.0    1
22.0    1
23.0    1
43.0    1
dtype: int64

In [25]:
# 4. Filtering and Grouping Data
dfg = df.groupby('Host city').mean() # mean() = average
# print(dfg) # Host city (index) / East / West / Diff
size = df.groupby('Host city').size()
dfg["Count"] = size # adding a "Count" column to the DF
dfg = dfg.sort_values('Count')
dfg = dfg[dfg['Count'] > 1]

## question 2. If a city hosted a game more than once, what was the average score 
## for the Eastern Conference team and the Western Conference team? 
print("Cities who hosted the game more than once:")
dfg

Cities who hosted the game more than once:


,East,West,Diff,Count
Host city,,,,
Atlanta,139.000000,140.000000,9.000000,2
Seattle,136.000000,144.000000,8.000000,2
Chicago,121.000000,108.500000,12.500000,2
Inglewood,121.000000,117.500000,5.500000,2
Orlando,131.000000,152.500000,21.500000,2
Denver,139.500000,130.000000,9.500000,2
Houston,131.333333,135.333333,5.333333,3
New Orleans,159.666667,158.333333,8.000000,3
St. Louis,128.000000,130.333333,11.000000,3


In [ ]:
# end